In [ ]:
# Imports
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# Paths
base_dir = "/kaggle/input/dagm-2007-competition-dataset-optical-inspection/DAGM_KaggleUpload"
classes = ["Class" + str(i+1) for i in range(10)]
dev_classes = classes[:6]
comp_classes = classes[6:]
print(dev_classes)
print(comp_classes)

In [ ]:
# Create lists with paths for training and test images (with labeled defect images)
trn_imdir  = []  
trn_ims    = []
trn_labdir = []
trn_labims = []

tst_imdir  = []
tst_ims    = []
tst_labdir = []
tst_labims = []

for cls in classes:
    # Get file names for all training images and label images
    trn_imdir.append(os.path.join(base_dir, cls, "Train"))    
    trn_ims.append(sorted(glob.glob(trn_imdir[-1] + "/*.PNG"))) 
    trn_labdir.append(os.path.join(base_dir, cls, "Train", "Label"))   
    trn_labims.append(sorted(glob.glob(trn_labdir[-1] + "/*.PNG")))
    
    tst_imdir.append(os.path.join(base_dir, cls, "Test"))
    tst_ims.append(sorted(glob.glob(tst_imdir[-1] + "/*.PNG")))    
    tst_labdir.append(os.path.join(base_dir, cls, "Test", "Label"))   
    tst_labims.append(sorted(glob.glob(tst_labdir[-1] + "/*.PNG")))



In [ ]:
print(trn_imdir[5])

In [ ]:
import matplotlib.pyplot as plt

# Misalkan trn_ims dan trn_labims sudah diisi dengan gambar dan label
# Pastikan Anda memiliki gambar dan label yang sesuai

# Menampilkan gambar contoh dengan labelnya
num_classes = min(len(dev_classes), len(trn_ims))  # Menghindari IndexError

for i in range(num_classes):
    if trn_ims[i]:  # Pastikan ada gambar
        example_defect_image = os.path.basename(trn_labims[i][0])
        im = plt.imread(trn_ims[i][0])  # Load the first training image
        labim = plt.imread(trn_labdir[i] + "/" + example_defect_image)  # Load the corresponding label image
        
        plt.figure(figsize=(12, 9))
        plt.subplot(121)
        plt.imshow(im, cmap="gray")
        plt.title(f"Image from {dev_classes[i]}")
        plt.axis('off')
        
        plt.subplot(122)
        plt.imshow(labim, cmap="gray")
        plt.title(f"Label for {dev_classes[i]}")
        plt.axis('off')
        
        plt.show()  # Show the figure
    else:
        print(f"No training images found for {dev_classes[i]}.")


In [ ]:
# Function to load and preprocess images
def load_and_preprocess_images(image_paths, label_paths, target_size=(256, 256)):
    images = []
    labels = []
    
    for img_path, lab_path in zip(image_paths, label_paths):
        if os.path.exists(img_path) and os.path.exists(lab_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, target_size)
            img = img / 255.0  # Normalize to [0, 1]
            images.append(img)

            lab = cv2.imread(lab_path, cv2.IMREAD_GRAYSCALE)
            lab = cv2.resize(lab, target_size)
            lab = lab / 255.0  # Normalize to [0, 1]
            labels.append(lab)
    
    return np.array(images), np.array(labels)

# Load training and testing data
X_train, y_train = [], []
X_test, y_test = [], []

for i in range(len(dev_classes)):
    if trn_ims[i]:  # Ensure there are training images
        X, y = load_and_preprocess_images(trn_ims[i], trn_labims[i])
        X_train.append(X)
        y_train.append(y)

    if tst_ims[i]:  # Ensure there are testing images
        X, y = load_and_preprocess_images(tst_ims[i], tst_labims[i])
        X_test.append(X)
        y_test.append(y)

# Concatenate all classes
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)
X_test = np.concatenate(X_test)
y_test = np.concatenate(y_test)

# Reshape labels for binary segmentation
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

# Define the AIS-Net model
def create_ais_net(input_shape):
    inputs = layers.Input(shape=input_shape)

    # MSDC Module
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # DACG Module
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # ACP Module
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)  # Output layer for binary segmentation
    outputs = layers.UpSampling2D((8, 8))(x)  # Upsampling to original size

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Create and train the model
input_shape = (256, 256, 1)  # Adjust based on your preprocessing
model = create_ais_net(input_shape)

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Optionally, visualize some predictions
def visualize_predictions(X, y, model, num_images=5):
    indices = np.random.choice(len(X), num_images, replace=False)
    for i in indices:
        pred = model.predict(X[i:i+1])
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 3, 1)
        plt.imshow(X[i].squeeze(), cmap='gray')
        plt.title("Input Image")
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(y[i].squeeze(), cmap='gray')
        plt.title("True Label")
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(pred.squeeze(), cmap='gray')
        plt.title("Predicted Label")
        plt.axis('off')
        
        plt.show()

# Visualize predictions
visualize_predictions(X_test, y_test, model)